### Loading triads and receptive, generative baseline data

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import os
from time import sleep
from tqdm import tqdm
ROOTPATH = os.getcwd()[:-3]

In [43]:
triads = pd.read_csv(ROOTPATH + "data\\wikipedia_triads_sequential_edgelist\\wikipedia_triads_sequential_edgelist.csv",
                    index_col=0)
triads.head(3)

,node_1,node_2,node_3,first_link_node_1,first_link_node_2,first_link_sign,second_link_node_1,second_link_node_2,second_link_sign,third_link_node_1,...,third_link_sign,fourth_link_node_1,fourth_link_node_2,fourth_link_sign,fifth_link_node_1,fifth_link_node_2,fifth_link_sign,sixth_link_node_1,sixth_link_node_2,sixth_link_sign
0,2762.0,2349.0,2431.0,2349.0,2431.0,1.0,2762.0,2431.0,1.0,2349.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2466.0,5051.0,757.0,757.0,2466.0,1.0,2466.0,757.0,1.0,2466.0,...,1.0,757.0,5051.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1673.0,52.0,863.0,1673.0,863.0,1.0,1673.0,52.0,1.0,863.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
rec_gen_baselines = pd.read_csv(ROOTPATH + "data\\preprocessed\\generative_receptive_wikipedia.csv", index_col=0)
rec_gen_baselines.head(3)

,node,type,generative_count,generative_probability,receptive_count,receptive_probability
0,1,1,20.0,1.00,27.0,0.900000
1,1,-1,0.0,0.00,3.0,0.100000
2,2,1,3.0,0.75,22.0,0.956522


#### Filtering out only those triads in the df for which the sets of two nodes for the first three links are different (i.e. become a triad after the first three linkages)

In [46]:
node_sets=triads[['first_link_node_1','first_link_node_2',
                'second_link_node_1','second_link_node_2',
                'third_link_node_1','third_link_node_2']].apply(set,axis=1)
triads=triads.loc[[len(set)==3 for set in node_sets],]

In [47]:
eventual_triads_indices=[]

list_row_sets=[]

for index, row in tqdm(triads.iterrows()):
    
    if len(set([frozenset(tuple([row['first_link_node_1'],row['first_link_node_2']])),
               frozenset(tuple([row['second_link_node_1'],row['second_link_node_2']])),
               frozenset(tuple([row['third_link_node_1'],row['third_link_node_2']]))]))==3:
        
        eventual_triads_indices.append(index)
        
triads=triads.filter(eventual_triads_indices,axis=0)

607279it [00:40, 14967.80it/s]


In [48]:
triads.head(3)

,node_1,node_2,node_3,first_link_node_1,first_link_node_2,first_link_sign,second_link_node_1,second_link_node_2,second_link_sign,third_link_node_1,...,third_link_sign,fourth_link_node_1,fourth_link_node_2,fourth_link_sign,fifth_link_node_1,fifth_link_node_2,fifth_link_sign,sixth_link_node_1,sixth_link_node_2,sixth_link_sign
0,2762.0,2349.0,2431.0,2349.0,2431.0,1.0,2762.0,2431.0,1.0,2349.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1673.0,52.0,863.0,1673.0,863.0,1.0,1673.0,52.0,1.0,863.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,698.0,2091.0,1934.0,698.0,2091.0,1.0,2091.0,1934.0,1.0,698.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
triads=triads.reset_index(drop=True)

In [105]:
triads.to_csv('wikipedia_triads_sequential_edgelist.csv')

### Defining and assigning triad types:

In [49]:
# triad_groups
t1=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['second_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))
t2=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['second_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))
t3=((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))
t4=((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))|((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))
t5=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['second_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))
t6=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['second_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))
t7=((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))|((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))
t8=((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))|((triads['first_link_node_2']==triads['second_link_node_2'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))
t9=((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))
t10=((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))|((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))
t11=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==1))
t12=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))
t13=((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))|((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_node_2']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))
t14=((triads['first_link_node_1']==triads['second_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))
t15=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==1)&(triads['second_link_sign']==-1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==1))
t16=((triads['first_link_node_2']==triads['second_link_node_1'])&(triads['first_link_node_1']==triads['third_link_node_2'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))|((triads['first_link_node_1']==triads['second_link_node_2'])&(triads['first_link_node_2']==triads['third_link_node_1'])&(triads['first_link_sign']==-1)&(triads['second_link_sign']==-1))

In [50]:
triads.loc[t1,'triad_context']='t1'
triads.loc[t2,'triad_context']='t2'
triads.loc[t3,'triad_context']='t3'
triads.loc[t4,'triad_context']='t4'
triads.loc[t5,'triad_context']='t5'
triads.loc[t6,'triad_context']='t6'
triads.loc[t7,'triad_context']='t7'
triads.loc[t8,'triad_context']='t8'
triads.loc[t9,'triad_context']='t9'
triads.loc[t10,'triad_context']='t10'
triads.loc[t11,'triad_context']='t11'
triads.loc[t12,'triad_context']='t12'
triads.loc[t13,'triad_context']='t13'
triads.loc[t14,'triad_context']='t14'
triads.loc[t15,'triad_context']='t15'
triads.loc[t16,'triad_context']='t16'

Distribution of triad types for figure 2

In [88]:
wiki_count_context=triads.groupby('triad_context')['node_1'].count().reset_index()
wiki_count_context.columns=['triad_context','count']
wiki_count_context

,triad_context,count
0,t1,95716
1,t10,19053
2,t11,6543
3,t12,1169
4,t13,12887
5,t14,6803
6,t15,1348
7,t16,237
8,t2,16964
9,t3,176701


Probability of positive third links

In [222]:
triad_context_third_link_sign = triads.groupby(['triad_context','third_link_sign'])['node_1'].count().reset_index()
triad_context = triads.groupby(['triad_context'])['node_2'].count().reset_index()
p_positive=triad_context_third_link_sign.merge(triad_context,how='left',on='triad_context')
p_positive.columns=['triad_context','third_link_sign','count_pos_neg','count_all']
p_positive=p_positive.loc[p_positive['third_link_sign']==1]
p_positive['p_positive_edge']=p_positive['count_pos_neg']/p_positive['count_all']
p_positive=p_positive[['triad_context','p_positive_edge']]
p_positive

,triad_context,p_positive_edge
1,t1,0.908375
3,t10,0.575657
5,t11,0.847165
7,t12,0.852866
9,t13,0.849461
11,t14,0.654417
13,t15,0.839763
15,t16,0.873418
17,t2,0.546746
19,t3,0.902247


Receptive and generative surprise

In [255]:
# pos rec and gen probs:
pos_receptive_probs=rec_gen_baselines.loc[rec_gen_baselines['type']==1][['node','receptive_probability']]
pos_generative_probs=rec_gen_baselines.loc[rec_gen_baselines['type']==1][['node','generative_probability']]

# attaching receptive surprise to third linkage in triads df:
rec_baselines=triads[['triad_context','third_link_node_2']]
#rec_baselines=rec_baselines.merge(pos_receptive_probs,how='left',left_on='third_link_node_2',right_on='node').groupby(['triad_context'])['receptive_probability'].aggregate(['mean','std']).reset_index()


In [256]:
rec_baselines.merge(pos_receptive_probs,how='left',left_on='third_link_node_2',right_on='node').groupby(['triad_context'])['receptive_probability'].aggregate(['mean','std','count']).reset_index()

,triad_context,mean,std,count
0,t1,0.878442,0.146814,95716
1,t10,0.608537,0.223658,18808
2,t11,0.841387,0.187480,6543
3,t12,0.831147,0.206064,1169
4,t13,0.847610,0.157803,12887
5,t14,0.634520,0.216381,6735
6,t15,0.834596,0.201128,1348
7,t16,0.810635,0.202326,237
8,t2,0.584425,0.239523,16662
9,t3,0.858471,0.178432,176701


In [ ]:
rec_baselines.columns=['triad_context','pos_receptive_baseline_third_to_node','pos_receptive_baseline_std_third_to_node']
rec_baselines=rec_baselines.merge(p_positive,how='inner',on='triad_context')
rec_baselines['receptive_surprise']=(rec_baselines['p_positive_edge']-rec_baselines['pos_receptive_baseline_third_to_node'])/rec_baselines['pos_receptive_baseline_std_third_to_node']

In [244]:
# attaching generative surprise to third linkage in triads df:
gen_baselines=triads[['triad_context','third_link_node_1']]
gen_baselines=gen_baselines.merge(pos_generative_probs,how='left',left_on='third_link_node_1',right_on='node').groupby(['triad_context'])['generative_probability'].aggregate(['mean','std']).reset_index()
gen_baselines.columns=['triad_context','pos_generative_baseline_third_from_node','pos_generative_baseline_std_third_from_node']
gen_baselines=gen_baselines.merge(p_positive,how='inner',on='triad_context')
gen_baselines['generative_surprise']=(gen_baselines['p_positive_edge']-gen_baselines['pos_generative_baseline_third_from_node'])/gen_baselines['pos_generative_baseline_std_third_from_node']

In [246]:
# merge
surprise=rec_baselines.merge(gen_baselines,how='inner',on='triad_context')[['triad_context','generative_surprise','receptive_surprise']]
surprise.head(3)

,triad_context,generative_surprise,receptive_surprise
0,t1,0.558971,0.203881
1,t10,-1.128794,-0.147011
2,t11,0.426062,0.030820


Merging all results

In [245]:
gen_baselines.head(1)

,triad_context,pos_generative_baseline_third_from_node,pos_generative_baseline_std_third_from_node,p_positive_edge,generative_surprise
0,t1,0.838022,0.125861,0.908375,0.558971


In [ ]:
gen_baselines.head(3)

In [247]:
wiki_count_context.head(1)

,triad_context,count
0,t1,95716


In [248]:
p_positive.head(1)

,triad_context,p_positive_edge
1,t1,0.908375


In [249]:
surprise.head(1)

,triad_context,generative_surprise,receptive_surprise
0,t1,0.558971,0.203881


In [250]:
results=wiki_count_context.merge(p_positive,how='inner',on='triad_context')
results=results.merge(surprise,how='inner',on='triad_context')
results

,triad_context,count,p_positive_edge,generative_surprise,receptive_surprise
0,t1,95716,0.908375,0.558971,0.203881
1,t10,19053,0.575657,-1.128794,-0.147011
2,t11,6543,0.847165,0.426062,0.030820
3,t12,1169,0.852866,0.450071,0.105397
4,t13,12887,0.849461,0.460254,0.011725
5,t14,6803,0.654417,-0.574347,0.091954
6,t15,1348,0.839763,0.463465,0.025687
7,t16,237,0.873418,0.606496,0.310306
8,t2,16964,0.546746,-1.817228,-0.157309
9,t3,176701,0.902247,0.589543,0.245336


adding checks for balance and status model fit

In [262]:
theory_definition=pd.DataFrame([['t1',1,1,1,1],
                                ['t2',-1,-1,-1,1],
                                ['t5',-1,-1,1,-1],
                                ['t6',1,1,-1,-1],
                                ['t3',-1,-1,-1,1],
                                ['t4',-1,-1,1,1],
                                ['t7',-1,-1,-1,-1],
                                ['t8',1,1,1,-1],
                                ['t9',-1,-1,1,-1],
                                ['t10',-1,-1,-1,-1],
                                ['t13', -1,-1,1,1],
                                ['t14',1,1,-1,1],
                                ['t11',1,1,-1,-1],
                                ['t12',-1,-1,1,-1],
                                ['t15',-1,-1,-1,1],
                                ['t16',1,1,1,1]], columns=['triad_context','bal_gen_sign','bal_rec_sign','stat_gen_sign','stat_rec_sign'])
theory_definition.head(3)

,triad_context,bal_gen_sign,bal_rec_sign,stat_gen_sign,stat_rec_sign
0,t1,1,1,1,1
1,t2,-1,-1,-1,1
2,t5,-1,-1,1,-1


In [264]:
results=results.merge(theory_definition,how='inner',on='triad_context')

In [271]:
# balance theory
results['Bg']=np.sign(results['generative_surprise'])==results['bal_gen_sign']
results['Br']=np.sign(results['receptive_surprise'])==results['bal_rec_sign']

# status theory
results['Sg']=np.sign(results['generative_surprise'])==results['stat_gen_sign']
results['Sr']=np.sign(results['receptive_surprise'])==results['stat_rec_sign']

In [274]:
results=results[['triad_context','count','p_positive_edge','generative_surprise','receptive_surprise','Bg','Br','Sg','Sr']]

In [276]:
results

,triad_context,count,p_positive_edge,generative_surprise,receptive_surprise,Bg,Br,Sg,Sr
0,t1,95716,0.908375,0.558971,0.203881,True,True,True,True
1,t10,19053,0.575657,-1.128794,-0.147011,True,True,True,True
2,t11,6543,0.847165,0.426062,0.030820,True,True,False,False
3,t12,1169,0.852866,0.450071,0.105397,False,False,True,False
4,t13,12887,0.849461,0.460254,0.011725,False,False,True,True
5,t14,6803,0.654417,-0.574347,0.091954,False,True,True,True
6,t15,1348,0.839763,0.463465,0.025687,False,False,False,True
7,t16,237,0.873418,0.606496,0.310306,True,True,True,True
8,t2,16964,0.546746,-1.817228,-0.157309,True,True,True,False
9,t3,176701,0.902247,0.589543,0.245336,False,False,False,True


In [279]:
results.apply('sum',axis=0)

triad_context          t1t10t11t12t13t14t15t16t2t3t4t5t6t7t8t9
count                                                   525032
p_positive_edge                                        12.1909
generative_surprise                                   0.588352
receptive_surprise                                    -1.19201
Bg                                                           6
Br                                                           8
Sg                                                          12
Sr                                                          12
dtype: object

In [277]:
results.to_csv('Figure_2_results.csv')